In [16]:
import pandas as pd
from datetime import date, timedelta

In [17]:
file_path = "https://www.dhhs.vic.gov.au/ncov-covid-cases-by-age-group-csv"
df = pd.read_csv(file_path)

In [18]:
df.tail()

,diagnosis_date,agegroup
676107,2022-03-06,60-69
676108,2022-03-06,50-59
676109,2022-03-06,40-49
676110,2022-03-06,10-19_
676111,2022-03-06,10-19_


In [19]:
df_pivot = pd.pivot_table(df, index = 'diagnosis_date', columns = 'agegroup', 
                         values = 'agegroup', aggfunc = len, fill_value = 0)

In [20]:
df_pivot.tail()

agegroup,0-9,10-19_,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+
diagnosis_date,,,,,,,,,,
2022-03-02,258,542,505,364,381,230,111,70,25,10
2022-03-03,295,498,454,362,357,213,125,67,25,4
2022-03-04,223,432,324,303,312,187,124,63,20,5
2022-03-05,192,435,293,260,287,205,99,60,28,2
2022-03-06,163,290,190,189,232,129,92,73,23,7


In [21]:
# check that the latest date's sum matches the number of cases reported by Vic govt
# the latest row's date is always yesterday

yesterday = date.today() - timedelta(days = 1)

# the dates in the pivot table are strings
# so use isoformat() to convert date into string before looking it up in the table
df_pivot.loc[yesterday.isoformat()].sum()

1388

In [22]:
# take only the 0-9 and 10-19 cases, from 5 Dec onwards
# for each date, store the previous date's cases; this matches what I've done for NSW
# eg. on 8 Dec, the Vic cases are those reported on the 7th

first = date(2021, 12, 5)
today = date.today()

df_0_19 = pd.DataFrame(columns = ['date', 'cases_0_9', 'cases_10_19'])

curr = first
prev = first - timedelta(days = 1)

while (curr < today):
    
        df_0_19 = df_0_19.append({'date': curr.isoformat(), 'cases_0_9': df_pivot.loc[prev.isoformat(), '0-9'], 
                                  'cases_10_19': df_pivot.loc[prev.isoformat(), '10-19_']}, ignore_index = True)
        prev = curr
        curr = curr + timedelta(days = 1)

In [23]:
df_0_19.tail()

,date,cases_0_9,cases_10_19
88,2022-03-03,258,542
89,2022-03-04,295,498
90,2022-03-05,223,432
91,2022-03-06,192,435
92,2022-03-07,163,290


In [24]:
df_0_19.to_csv("../data/cases_0_19.csv")